# Which findmax implementation is the fastest? 

Julia has its own findmax function. However during the testing it seems a bit slow. So this notebook is tries to find the fastest implementation. 

I am using JULIA_NUM_THREADS=16 to see if multithreading can help speeding it up. 

In [15]:
using Base.Threads 

In [16]:
# This function uses purely julia findmax. It does not take advantage of multithreading.
function findmax_v1(lod::AbstractArray{<:Real,2}, dims::Int)
    res = findmax(lod, dims=dims)
    # get the first element, which is the max of the first dimension, and turn it into a column
    max = res[1]
    # get the second element, which is the cartisian index, and only get the first index of the tuple(cartisian index), and turn it into column
    maxidx = getindex.(res[2], 2)
    return hcat(maxidx, max)
end


findmax_v1 (generic function with 1 method)

In [42]:
# This function uses fused approach. Multhreading implemented at top level. 
function findmax_v2(lod::AbstractArray{<:Real,2}, dims::Int)
    arraydim = 0 
    if dims == 1
        arraydim = 2
    else
        arraydim = 1
    end
    max_array = Array{typeof(lod[1,1]),2}(undef, size(lod, arraydim), 2)
    Threads.@threads for i in 1:size(lod, arraydim)
        if dims == 1 
            (max, index) = findmax(lod[:, i])
        elseif dims == 2
            (max, index) = findmax(lod[i, :])
        else 
            error("dims must be 1 or 2")
        end
        max_array[i,1] = convert(typeof(lod[1,1]), index)
        max_array[i,2] = max
    end
end

findmax_v2 (generic function with 1 method)

In [18]:
# This function is purely hand written, a good old nested for loop. It only findmax across dim=2
function findmax_v3(lod::AbstractArray{<:Real,2})
    max_array = Array{typeof(lod[1,1]),2}(undef, size(lod)[1], 2)
    Threads.@threads for i in 1:size(lod)[1]
        # for i in 1:size(lod)[1]
        temp = lod[i, 1]
        idx = 1
        for j in 2:size(lod)[2]
            if temp < lod[i,j]
                temp = lod[i,j]
                idx = j
            end
        end
        max_array[i,1] = idx
        max_array[i,2] = temp
    end
    return max_array
end

findmax_v3 (generic function with 1 method)

In [19]:
a = rand(20000, 10000);

In [55]:
@time v1 = findmax_v1(a,2)
@time v2 = findmax_v2(a,2)
@time v3 = findmax_v3(a);

  0.895861 seconds (13 allocations: 937.906 KiB)
  0.410285 seconds (40.20 k allocations: 1.492 GiB)
  0.125187 seconds (198 allocations: 333.672 KiB)


### findmax_v4 
this version uses Distributed package rather than Threads. It is equivalent to findmax_v2, only difference is findmax_v2 uses Threads.@threads

In [57]:
using Distributed
using SharedArrays
# addprocs(16)
res = SharedArray{Float64}(size(a,1), 2)
@everywhere using Distributed   
nprocs()

17

In [59]:
s = time_ns()
@sync @distributed for row = 1:size(a,1)
    res[row,1] = findmax(a[row, :])[2]# Index
    res[row,2] = findmax(a[row, :])[1]# Max
end
e = time_ns()
println("distributed takes $((e-s)*0.000000001) seconds")

distributed takes 0.729388303 seconds


# comparing speed across rows ( column major access)
This compares @threads and @distributed on row major access.( memory access pattern is inefficient since Julia is using column major). 

In [60]:
res = SharedArray{Float64}(size(a,2), 2)
@everywhere using Distributed   
nprocs()
s = time_ns()
@sync @distributed for col = 1:size(a,2)
    res[col,1] = findmax(a[:, col])[2]# Index
    res[col,2] = findmax(a[:, col])[1]# Max
end
e = time_ns()
println("distributed takes $((e-s)*0.000000001) seconds")

distributed takes 0.33528427 seconds


In [61]:
@time v1 = findmax_v1(a,1)
@time v2 = findmax_v2(a,1)

  0.828353 seconds (8 allocations: 469.062 KiB)
  0.270993 seconds (20.20 k allocations: 1.491 GiB)


# Conclusion:

The conclusion is the the hand written nested for loop wins when using 16 threads. 
